In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-03-23 01:32:49,290 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.2.0.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/working


In [6]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()}

graphnet: INFO     2023-03-23 01:32:53 - get_logger - Writing log to logs/graphnet_20230323-013253.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-23 01:32:54.209883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 01:32:54.384372: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the

In [9]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-03-23 01:48:55,964 [INFO][utils] Output directory: output/2023-03-23_01-48-55
2023-03-23 01:48:55,964 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 01:48:55,964 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 01:48:55,965 [INFO][utils] Fix seed: 440
2023-03-23 01:48:56,347 [INFO][15-validation] Base score: 1.0182592239285344
2023-03-23 01:48:56,358 [INFO][15-validation] Num of low sigma events: 51592, Num of high sigma events: 148408
2023-03-23 01:48:56,407 [INFO][15-validation] Low sigma score: 0.20875941665868464, High sigma score: 1.2996706980853592
2023-03-23 01:48:56,676 [INFO][15-validation] Done.


In [17]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()} training_params.stage2=True

graphnet: INFO     2023-03-23 02:23:16 - get_logger - Writing log to logs/graphnet_20230323-022316.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-23 02:23:17.683995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 02:23:17.910869: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the

In [18]:
!python 15-validation.py settings.in_kaggle={in_kaggle()} training_params.stage2=True

2023-03-23 02:26:23,901 [INFO][utils] Output directory: output/2023-03-23_02-26-23
2023-03-23 02:26:23,902 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 02:26:23,902 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 02:26:23,902 [INFO][utils] Fix seed: 440
2023-03-23 02:26:24,005 [INFO][15-validation] Base score: 0.2092210746124225
2023-03-23 02:26:24,007 [INFO][15-validation] Num of low sigma events: 51241, Num of high sigma events: 351
2023-03-23 02:26:24,026 [INFO][15-validation] Low sigma score: 0.20530310216450823, High sigma score: 0.7811892404345699
2023-03-23 02:26:24,026 [INFO][15-validation] Done.


In [19]:
!python 20-ensemble.py settings.in_kaggle={in_kaggle()}

2023-03-23 02:26:38,841 [INFO][utils] Output directory: output/2023-03-23_02-26-38
2023-03-23 02:26:38,841 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 02:26:38,841 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 02:26:38,841 [INFO][utils] Fix seed: 440
2023-03-23 02:26:39,345 [INFO][20-ensemble] Base score: 1.0178703348037772
2023-03-23 02:26:39,371 [INFO][20-ensemble] Num of low sigma events: 51412, Num of high sigma events: 148588
2023-03-23 02:26:39,418 [INFO][20-ensemble] Low sigma score: 0.2085757324173632, High sigma score: 1.2990101573728978
2023-03-23 02:26:39,418 [INFO][20-ensemble] Done.
